# SageMaker Coding Agent - Setup

This notebook helps you:
1. Discover available Bedrock models
2. Check permissions
3. Configure the agent
4. Index your codebase (optional)

## 1. Install Dependencies

In [ ]:
# Install required packages
!pip install -q boto3 ipywidgets pandas openpyxl python-docx Pillow numpy

## 2. Discover Bedrock Models

In [ ]:
import boto3
import json
from botocore.exceptions import ClientError

def discover_bedrock_models(region: str = "ap-southeast-2"):
    """Discover available Bedrock Claude models and check permissions."""
    
    runtime = boto3.client("bedrock-runtime", region_name=region)
    
    results = {
        "region": region,
        "available_models": [],
        "permission_issues": [],
        "recommended_model": None
    }
    
    # Claude models to check (newest first)
    claude_models = [
        ("anthropic.claude-3-5-sonnet-20241022-v2:0", "Claude 3.5 Sonnet v2"),
        ("anthropic.claude-3-5-sonnet-20240620-v1:0", "Claude 3.5 Sonnet"),
        ("anthropic.claude-3-opus-20240229-v1:0", "Claude 3 Opus"),
        ("anthropic.claude-3-sonnet-20240229-v1:0", "Claude 3 Sonnet"),
        ("anthropic.claude-3-haiku-20240307-v1:0", "Claude 3 Haiku"),
    ]
    
    print(f"Checking Bedrock models in region: {region}\n")
    
    for model_id, display_name in claude_models:
        try:
            # Test invoke with minimal tokens
            response = runtime.invoke_model(
                modelId=model_id,
                body=json.dumps({
                    "anthropic_version": "bedrock-2023-05-31",
                    "max_tokens": 10,
                    "messages": [{"role": "user", "content": "Hi"}]
                }),
                contentType="application/json"
            )
            results["available_models"].append({
                "id": model_id,
                "name": display_name,
                "status": "available"
            })
            print(f"✅ {display_name}: Available")
            
            if results["recommended_model"] is None:
                results["recommended_model"] = model_id
                
        except ClientError as e:
            error_code = e.response["Error"]["Code"]
            if error_code == "AccessDeniedException":
                results["permission_issues"].append({
                    "model": model_id,
                    "error": "Access denied - model not enabled"
                })
                print(f"❌ {display_name}: Access denied (enable in Bedrock console)")
            elif error_code == "ValidationException":
                results["permission_issues"].append({
                    "model": model_id,
                    "error": f"Not available in {region}"
                })
                print(f"⚠️ {display_name}: Not available in {region}")
            else:
                print(f"❌ {display_name}: {error_code}")
    
    return results

# Run discovery (change region if needed)
results = discover_bedrock_models("ap-southeast-2")

## 3. Permission Setup Instructions

In [ ]:
def print_setup_instructions(results):
    if results["permission_issues"]:
        print("\n" + "="*60)
        print("PERMISSION SETUP REQUIRED")
        print("="*60)
        print("""
To enable Claude models in Bedrock:

1. Go to AWS Console → Amazon Bedrock
2. Click "Model access" in the left sidebar
3. Click "Manage model access"
4. Select the Claude models you want to use:
   - Claude 3.5 Sonnet (recommended)
   - Claude 3 Haiku (for fast tasks)
5. Click "Request model access"
6. Wait for approval (usually instant for Claude)
7. Re-run this cell to verify

Note: Make sure your SageMaker execution role has:
- bedrock:InvokeModel
- bedrock:InvokeModelWithResponseStream
""")
    else:
        print(f"\n✅ All models available! Recommended: {results['recommended_model']}")

print_setup_instructions(results)

## 4. Save Configuration

In [ ]:
from config import AgentConfig

# Create config with discovered model
config = AgentConfig(
    region=results["region"],
    primary_model=results["recommended_model"] or "anthropic.claude-3-sonnet-20240229-v1:0",
    workspace_root=".",
)

# Save configuration
config.save()

print(f"Configuration saved!")
print(f"  Region: {config.region}")
print(f"  Primary model: {config.primary_model}")
print(f"  Workspace: {config.workspace_root}")

## 5. (Optional) Index Codebase for Semantic Search

In [ ]:
# Uncomment and run to index your codebase for semantic search
# This enables natural language code search

# from core.semantic_search import SemanticSearch
# 
# searcher = SemanticSearch(region=config.region)
# num_chunks = searcher.index_codebase(".")  # Index current directory
# print(f"Indexed {num_chunks} code chunks")

## 6. Create Project Instructions (Optional)

Create an `AGENTS.md` file to give the agent project-specific instructions.

In [ ]:
# Uncomment to create a template AGENTS.md

# from core.project_config import ProjectConfig
# 
# path = ProjectConfig.create_template(".", "AGENTS.md")
# print(f"Created template: {path}")
# print("Edit this file to add project-specific instructions for the agent.")

## Done!

Setup is complete. Open `agent.ipynb` to start using the coding agent.